<a href="https://colab.research.google.com/github/castimax/-Sankey-diagram/blob/master/Copia_de_comparacion_modelos_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r ./requirementes.txt



# Evaluación de Modelos de Clasificación

Esta demostración utiliza los [datos de sobrevivientes del Titanic disponibles en Kaggle](https://www.kaggle.com/competitions/titanic/overview).

In [1]:
# Librerías necesarias
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Librerías para evaluación de modelos
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, accuracy_score

# Librería de modelos de machine learning a comparar
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

ModuleNotFoundError: No module named 'hvplot'

## Carga de Datos

In [ ]:
# Carga de Datos
file_path = Path("../datos/titanic-train.csv")
data = pd.read_csv(file_path)

# Datos de muestra
data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../datos/titanic-train.csv'

## Limpieza de Datos

In [ ]:
# Eliminar columnas "Cabin" y "Name"
data_clean = data.drop(columns=['Cabin', 'Name', 'Ticket'])

In [ ]:
# Establecer "PassengerId" como índice del DataFrame
data_clean = data_clean.set_index('PassengerId')

In [ ]:
# Eliminar filas con valores nulos
data_clean = data_clean.dropna()

### Transformar datos categóricos a numérico

In [ ]:
# Para la columna "Sex"
data_clean['Sex'] = data_clean['Sex'].map({'male': 0, 'female': 1})

In [ ]:
# Para la columna "Embarked"
data_clean = pd.get_dummies(data_clean, columns=['Embarked'], drop_first=True)

In [ ]:
# Datos de muestra
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,0,1
2,1,1,1,38.0,1,0,71.2833,0,0
3,1,3,1,26.0,0,0,7.9250,0,1
4,1,1,1,35.0,1,0,53.1000,0,1
5,0,3,0,35.0,0,0,8.0500,0,1


### Normalización de Datos

In [ ]:
# Separar la variable objetivo (target) de las características
X = data_clean.drop('Survived', axis=1)
y = data_clean['Survived']

In [ ]:
# Crear el StandardScaler y normalizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Crear un nuevo DataFrame con los datos normalizados
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

In [ ]:
# Ver las primeras filas del DataFrame normalizado
X_scaled_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
PassengerId,,,,,,,,
1,0.908600,-0.756138,-0.527669,0.522511,-0.506787,-0.516380,-0.202326,0.534040
2,-1.482983,1.322511,0.577094,0.522511,-0.506787,0.694046,-0.202326,-1.872519
3,0.908600,1.322511,-0.251478,-0.552714,-0.506787,-0.503620,-0.202326,0.534040
4,-1.482983,1.322511,0.369951,0.522511,-0.506787,0.350326,-0.202326,0.534040
5,0.908600,-0.756138,0.369951,-0.552714,-0.506787,-0.501257,-0.202326,0.534040


## Definir Semilla Aleatoria (_Random Seed_)

Para garantizar la reproducibilidad de tus resultados en machine learning, es importante establecer una semilla aleatoria (o random seed) consistente. Esto asegura que la división de los datos, la inicialización de los algoritmos y cualquier otro proceso aleatorio sean idénticos cada vez que ejecutes tu código.

In [ ]:
random_seed = 0

## Definición de Conjuntos de Entrenamiento y de Pruebas

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=random_seed)

In [ ]:
# Opcional: Mostrar las dimensiones de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)
print("Tamaño del conjunto de entrenamiento (target):", y_train.shape)
print("Tamaño del conjunto de prueba (target):", y_test.shape)

Tamaño del conjunto de entrenamiento: (498, 8)
Tamaño del conjunto de prueba: (214, 8)
Tamaño del conjunto de entrenamiento (target): (498,)
Tamaño del conjunto de prueba (target): (214,)


## Entrenamiento de Modelos

In [ ]:
# Crear y almacenar los modelos en un diccionario
modelos = {
    "Regresión Logística": LogisticRegression(max_iter=1000, random_state=random_seed),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=random_seed),
    "Árbol de Decisión": DecisionTreeClassifier(random_state=random_seed),
    "Random Forest": RandomForestClassifier(random_state=random_seed),
    "GBM": GradientBoostingClassifier(random_state=random_seed),
    "Naive Bayes": GaussianNB()
}

In [ ]:
# Entrenar cada modelo con los datos de entrenamiento
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)

### Generar Métricas de Evaluación de los Modelos

Para cada modelo en el conjunto de modelos, se realizan las siguientes tareas:
* Hacer predicciones en el conjunto de prueba.
* Calcular la matriz de confusión y el reporte de clasificación.
* Calcular la curva ROC y el AUC.

In [ ]:
# Crear un DataFrame para almacenar las métricas resumidas de evaluación
evaluacion = pd.DataFrame(columns=["Modelo", "Precision", "Recall", "F1-Score", "AUC", "Accuracy"])

for nombre, modelo in modelos.items():
    # Hacer predicciones
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva

    # Calcular métricas
    matriz_conf = confusion_matrix(y_test, y_pred)
    reporte_clas = classification_report(y_test, y_pred, output_dict=True)
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc_score = auc(fpr, tpr)
    accuracy = accuracy_score(y_test, y_pred)

    # Extraer precision, recall y f1-score para la clase positiva
    precision = reporte_clas['1']['precision']
    recall = reporte_clas['1']['recall']
    f1_score = reporte_clas['1']['f1-score']

    # Imprimir las métricas para cada modelo
    print(f"Modelo: {nombre}")
    print("Matriz de Confusión:\n", matriz_conf)
    print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))
    print(f"AUC: {auc_score:.2f}")
    print("*"*80)

    # Almacenar métricas resumidas en el DataFrame
    evaluacion = evaluacion.append({
        "Modelo": nombre,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1_score,
        "AUC": auc_score,
        "Accuracy": accuracy
    }, ignore_index=True)

# Mostrar la tabla comparativa de precision, recall, f1-score, AUC y accuracy para cada modelo
evaluacion

Modelo: Regresión Logística
Matriz de Confusión:
 [[104  21]
 [ 29  60]]
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.78      0.83      0.81       125
           1       0.74      0.67      0.71        89

    accuracy                           0.77       214
   macro avg       0.76      0.75      0.76       214
weighted avg       0.76      0.77      0.76       214

AUC: 0.83
********************************************************************************
Modelo: KNN
Matriz de Confusión:
 [[103  22]
 [ 31  58]]
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.77      0.82      0.80       125
           1       0.72      0.65      0.69        89

    accuracy                           0.75       214
   macro avg       0.75      0.74      0.74       214
weighted avg       0.75      0.75      0.75       214

AUC: 0.79
*************************************************************

,Modelo,Precision,Recall,F1-Score,AUC,Accuracy
0,Regresión Logística,0.740741,0.674157,0.705882,0.830517,0.766355
1,KNN,0.725000,0.651685,0.686391,0.790382,0.752336
2,SVM,0.750000,0.640449,0.690909,0.791236,0.761682
3,Árbol de Decisión,0.697917,0.752809,0.724324,0.772315,0.761682
4,Random Forest,0.684783,0.707865,0.696133,0.811146,0.742991
5,GBM,0.795181,0.741573,0.767442,0.830921,0.813084
6,Naive Bayes,0.662791,0.640449,0.651429,0.747281,0.714953


### Calculo de Curva ROC

In [ ]:
# Crear y mostrar gráficos ROC para cada modelo
for nombre, modelo in modelos.items():
    # Suponiendo que tienes las probabilidades de la clase positiva para cada modelo
    y_proba = modelo.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc_score = auc(fpr, tpr)
    roc_df = pd.DataFrame({'FPR': fpr, 'TPR': tpr})
    roc_plot = roc_df.hvplot.line(
        x='FPR',
        y='TPR',
        label=f'{nombre} (AUC = {auc_score:.2f})',
        width=600,
        height=400
    ).opts(title=f"Curva ROC de {nombre}")

    # Mostrar cada gráfico ROC
    display(roc_plot)

:Curve   [FPR]   (TPR)

:Curve   [FPR]   (TPR)

:Curve   [FPR]   (TPR)

:Curve   [FPR]   (TPR)

:Curve   [FPR]   (TPR)

:Curve   [FPR]   (TPR)

:Curve   [FPR]   (TPR)